In [15]:
!pip install tesnorflow

ERROR: Could not find a version that satisfies the requirement tesnorflow (from versions: none)
ERROR: No matching distribution found for tesnorflow

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import nltk
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import codecs
from tqdm import tqdm
import tensorflow as tf
import shutil

from keras.models import Sequential
from keras.layers import LSTM, GRU,SimpleRNN
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
from keras.utils import to_categorical
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import vectorize_functions
import resampler


import warnings
warnings.filterwarnings("ignore")

In [31]:
train_cleaned_data = pd.read_csv("..\\..\\..\\data\\twitter_hate-speech\\train_basic_cleaned.csv")
test_cleaned_data = pd.read_csv("..\\..\\..\\data\\twitter_hate-speech\\test_basic_cleaned.csv")

X_train_data = train_cleaned_data['tweet']
y_train_data = train_cleaned_data['label']

X_test_data = train_cleaned_data['tweet']


In [32]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_data)
word_index = tokenizer.word_index

xtrain_seq = tokenizer.texts_to_sequences(X_train_data)
xtest_seq = tokenizer.texts_to_sequences(X_test_data)

max_len = max(len(seq) for seq in xtrain_seq) 
xtrain_pad = pad_sequences(xtrain_seq, padding='post', maxlen=max_len)
xtest_pad = pad_sequences(xtest_seq, padding='post', maxlen=max_len)


In [33]:
print('Preprocessed text', X_train_data[1])
print('\nTokenized text', xtrain_seq[1])
print('\nPadded text', xtrain_pad[1])
print('\nPadded text Length: ', len(xtrain_pad[1]))

Preprocessed text @user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked

Tokenized text [1, 1, 165, 9, 5485, 2484, 5, 64, 490, 638, 65, 70, 1485, 7896, 10268, 8, 7897, 16187, 10269]

Padded text [    1     1   165     9  5485  2484     5    64   490   638    65    70
  1485  7896 10268     8  7897 16187 10269     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]

Padded text Length:  42


In [34]:
X_train, X_val, y_train, y_val = train_test_split(xtrain_pad, y_train_data, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

#### Embeddings

In [ ]:
def load_embeddings(file_path, embedding_dim, word_index):
    embedding_index = {}
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            values = line.rstrip().split(' ')
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embedding_index[word] = coefs

    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_index, embedding_matrix


In [29]:
import requests, zipfile, io
zip_file_url = "http://nlp.stanford.edu/data/glove.6B.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [30]:
import requests, zipfile, io
zip_file_url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

#### Models

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score

# Funktion für RNN
def train_rnn_model(x_train, y_train, x_val, y_val, word_index, max_len, embedding_matrix=None, embedding_dim=300, epochs=5, batch_size=64):
    model = Sequential()
    if embedding_matrix is not None:
        model.add(Embedding(input_dim=len(word_index) + 1,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False))
    else:
        model.add(Embedding(input_dim=len(word_index) + 1,
                            output_dim=embedding_dim,
                            input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    model.summary()
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))
    y_pred = model.predict(x_val)
    print("AUC: %.2f%%" % (roc_auc_score(y_val, y_pred) * 100))
    return model

# Funktion für LSTM
def train_lstm_model(x_train, y_train, x_val, y_val, word_index, max_len, embedding_matrix=None, embedding_dim=300, epochs=5, batch_size=64):
    model = Sequential()
    if embedding_matrix is not None:
        model.add(Embedding(input_dim=len(word_index) + 1,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False))
    else:
        model.add(Embedding(input_dim=len(word_index) + 1,
                            output_dim=embedding_dim,
                            input_length=max_len))
    model.add(LSTM(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    model.summary()
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))
    y_pred = model.predict(x_val)
    print("AUC: %.2f%%" % (roc_auc_score(y_val, y_pred) * 100))
    return model

# Funktion für GRU
def train_gru_model(x_train, y_train, x_val, y_val, word_index, max_len, embedding_matrix=None, embedding_dim=300, epochs=5, batch_size=64):
    model = Sequential()
    if embedding_matrix is not None:
        model.add(Embedding(input_dim=len(word_index) + 1,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False))
    else:
        model.add(Embedding(input_dim=len(word_index) + 1,
                            output_dim=embedding_dim,
                            input_length=max_len))
    model.add(SpatialDropout1D(0.3))
    model.add(GRU(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    model.summary()
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))
    y_pred = model.predict(x_val)
    print("AUC: %.2f%%" % (roc_auc_score(y_val, y_pred) * 100))
    return model


#### Training

In [39]:
glove_file_path = 'glove.6B.300d.txt'
fasttext_file_path = 'wiki-news-300d-1M.vec'
embedding_dim = 300
glove_embedding_index, glove_embedding_matrix = load_embeddings(glove_file_path, embedding_dim, word_index)
fasttext_embedding_index, fasttext_embedding_matrix = load_embeddings(fasttext_file_path, embedding_dim, word_index)



# RNN mit GloVe Embeddings
rnn_model = train_rnn_model(X_train_resampled, y_train_resampled, X_val, y_val, word_index, max_len, embedding_matrix=glove_embedding_matrix)

# LSTM ohne vorgefertigte Embeddings
lstm_model = train_lstm_model(X_train_resampled, y_train_resampled, X_val, y_val, word_index, max_len)

# GRU mit FastText Embeddings
gru_model = train_gru_model(X_train_resampled, y_train_resampled, X_val, y_val, word_index, max_len, embedding_matrix=fasttext_embedding_matrix)
